In [1]:
!jar xvf images.zip

Streaming output truncated to the last 5000 lines.
 inflated: images/Rottweiler/n02106550_107.jpg
 inflated: images/Rottweiler/n02106550_10706.jpg
 inflated: images/Rottweiler/n02106550_10709.jpg
 inflated: images/Rottweiler/n02106550_10714.jpg
 inflated: images/Rottweiler/n02106550_10952.jpg
 inflated: images/Rottweiler/n02106550_10966.jpg
 inflated: images/Rottweiler/n02106550_11002.jpg
 inflated: images/Rottweiler/n02106550_11097.jpg
 inflated: images/Rottweiler/n02106550_11105.jpg
 inflated: images/Rottweiler/n02106550_11176.jpg
 inflated: images/Rottweiler/n02106550_11203.jpg
 inflated: images/Rottweiler/n02106550_11244.jpg
 inflated: images/Rottweiler/n02106550_11323.jpg
 inflated: images/Rottweiler/n02106550_11354.jpg
 inflated: images/Rottweiler/n02106550_11415.jpg
 inflated: images/Rottweiler/n02106550_11433.jpg
 inflated: images/Rottweiler/n02106550_11444.jpg
 inflated: images/Rottweiler/n02106550_11465.jpg
 inflated: images/Rottweiler/n02106550_11543.jpg
 inflated: images/Ro

In [2]:
from __future__ import print_function

import os
import argparse

import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from mpl_toolkits.axes_grid1 import ImageGrid
from scipy.optimize import linear_sum_assignment as linear_assignment_
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from itertools import chain

import torch
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torch.optim as optim
import torch.nn as nn
import torchvision

# Get data
from PIL import Image
import os.path
import shutil
import requests
import zipfile

from pathlib import Path
import glob
from sklearn.model_selection import train_test_split

In [3]:
from torch.autograd import Variable
import torch.nn.functional as F
# can use the below import should you choose to initialize the weights of your Net
import torch.nn.init as I

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:",device)

device: cuda


In [5]:
from os.path import join, expanduser

root = expanduser("")
imagesets = join(root, 'DATASETS', 'IMAGE')
models = join(root, 'Models')
plots = join(root, 'Plots')

In [6]:
import argparse # Importing the argparse module

def parse_args():
    """Parses command line arguments."""
    parser = argparse.ArgumentParser(description='PyTorch DML Training')
    parser.add_argument('--batch_size', help='Batch_size', default=64, type=int)
    parser.add_argument('--n_epochs', help='Number of Epochs', default=30, type=int)
    parser.add_argument('--learning_rate', help='Learning Rate', default=0.01, type=float)
    parser.add_argument('--saved_epoch', help='epoch of saved model', default=None, type=int)
    parser.add_argument('--run_id', help='Used to help identify artifacts', default=0, type=int)

    # Change: Removing the SystemExit call
    args, unknown = parser.parse_known_args()  # Parse arguments
    if unknown:  # If there are unknown arguments
        print(f"Warning: Ignoring unknown arguments: {unknown}")  # Print a warning

    return args

In [25]:
def train_model(batch_size, n_epochs, learning_rate,
                saved_epoch,
                run_id="def",
                save_every=1000, save_path=models,
                plot_every=500, plot_path=plots):
    # Setup save directories
    if save_path:
        save_path = os.path.join(save_path, f"run_{run_id}")
        os.makedirs(save_path, exist_ok=True)
    if plot_path:
        plot_path = os.path.join(plot_path, f"run_{run_id}")
        os.makedirs(plot_path, exist_ok=True)

    # Load network and use GPU
    net = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)

    # Get the number of input features for the classifier
    in_features = net.roi_heads.box_predictor.cls_score.in_features
    # Replace the pre-trained head with a new one (Note: `box_predictor` can be `FasterRCNNPredictor`)
    net.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, 120)

    if torch.cuda.is_available():
        net = net.cuda()
        cudnn.benchmark = True
    else:
        print("No GPU available")

    # Setup path to data folder
    image_path = Path("/content/classified_images/")

    # if the image folder doesn't exist, download it and prepare it...
    if image_path.is_dir():
        print(f"{image_path} directory exists.")
    else:
        print(f"Did not find {image_path} directory, creating one...")
        image_path.mkdir(parents=True, exist_ok=True)

    # Setup Dirs
    train_dir = image_path / "train"
    valid_dir = image_path / "valid"
    test_dir = image_path / "test"
    train_dir.mkdir(parents=True, exist_ok=True)
    valid_dir.mkdir(parents=True, exist_ok=True)
    test_dir.mkdir(parents=True, exist_ok=True)

    # Check if directories are populated with valid images
    def check_image_directories(root_dir):
        """Checks if image directories are populated and contain jpg formats."""
        for subdir in ['train', 'valid', 'test']:
            dir_path = os.path.join(root_dir, subdir)
            if not os.path.exists(dir_path):
                print(f"Directory {dir_path} does not exist. Creating it...")
                os.makedirs(dir_path)  # Create directory if it doesn't exist
                return False  # Return False to indicate directory was created
            if not os.listdir(dir_path):
                print(f"Directory {dir_path} is empty.")
                return False
            # Check if any images have valid extensions
            has_valid_images = glob.glob(os.path.join(dir_path,'**/*.jpg'), recursive=True)
            if not has_valid_images:
                print(f"Directory {dir_path} does not contain valid image files with supported extensions.")
                return False
            for item in has_valid_images:
                if os.path.isfile(item) and not item.lower().endswith('.jpg'):
                    print(f"File {item} has an unsupported format.")
                    return False
        print("Image directories are populated and contain supported image formats.")
        return True

    if not check_image_directories(str(image_path)):
        print("Please populate the image directories with valid images before running the training.")
        # Create symbolic links to example images if the train directory is empty
        if not os.listdir(train_dir):
            print(f"Creating symbolic links to example images in {train_dir}")
            example_images = [f for f in os.listdir(image_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
            # Check if example images exist before creating symlinks
            if example_images:
                for image in example_images:
                    source_path = os.path.join(image_path, image)
                    destination_path = os.path.join(train_dir, image)

                    # Handle existing symlinks
                    if os.path.islink(destination_path):
                        os.unlink(destination_path)

                    # Create the symbolic link
                    os.symlink(source_path, destination_path)

                print(f"Symbolic links created. Please replace them with your actual training images.")
            else:
                print("No example images found in the root directory.")
        return  # Exit the function to avoid further errors

    # transforms for images
    transforms = torchvision.transforms.Compose([
        torchvision.transforms.Resize(256),
        torchvision.transforms.CenterCrop(224),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.RandomHorizontalFlip(p = 0.5)
    ])

    # Load dataset
    train_data = ImageFolder(root=str(train_dir), transform=transforms)
    valid_data = ImageFolder(root=str(valid_dir), transform=transforms)
    test_data = ImageFolder(root=str(test_dir), transform=transforms)

    classes = train_data.classes
    print("Classes:", classes)
    print("Training data shape:", len(train_data))
    print("valid data shape:", len(valid_data))
    print("Testing data shape:", len(test_data))

    if len(train_data) == 0 or len(valid_data) == 0 or len(test_data) == 0:
        raise ValueError("One or more datasets are empty. Check your image directories.")

    #train_y, test_y = utils.get_labels(train_data), utils.get_labels(test_data)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    # obtain one batch of training images
    dataiter = iter(train_loader)
    images, labels = next(dataiter)
    images = np.swapaxes(np.swapaxes(images.numpy(), 1, 2), 2, 3)

    # plot the images in the batch, along with the corresponding labels
    fig = plt.figure(figsize=(batch_size/4+5, batch_size/4+5))
    for idx in np.arange(batch_size):
        ax = fig.add_subplot(batch_size//8, 8, idx+1, xticks=[], yticks=[])
        ax.imshow(images[idx])
        ax.set_title(classes[labels[idx]], {'fontsize': batch_size/5}, pad=0.4)
    plt.tight_layout(pad=1, w_pad=0, h_pad=0)
    if plot_path:
        plt.savefig(os.path.join(plot_path, "Initial_Visualization"))
    else:
        plt.show()
    plt.clf()

    # cross entropy loss combines softmax and nn.NLLLoss() in one single class.
    criterion = nn.NLLLoss()

    # stochastic gradient descent with a small learning rate
    optimizer = optim.SGD(net.parameters(), lr=learning_rate)

    # ToDo: Add to utils
    # Calculate accuracy before training
    correct = 0
    total = 0

    # Iterate through test dataset
    for images, labels in test_loader:
        images, labels = images.cuda(), labels.cuda()

        # set the model to evaluation mode before inference
        net.eval()

        # forward pass to get outputs
        # the outputs are a series of class scores
        # Since we are in evaluation mode, we don't need targets
        with torch.no_grad():  # Disable gradient calculation during inference
            outputs = net(images)

        # Set the model back to training mode
        net.train()

        # get the predicted class from the maximum value in the output-list of class scores
        _, predicted = torch.max(outputs.data, 1)

        # count up total number of correct labels
        # for which the predicted and true labels are equal
        total += labels.size(0)
        correct += (predicted == labels).sum()

    # calculate the accuracy
    # to convert `correct` from a Tensor into a scalar, use .item()
    accuracy = 100.0 * correct.item() / total

    print('Accuracy before training: ', accuracy)

    # Within the `train` function:
    def train(n_epochs):
        net.train()
        loss_over_time = [] # to track the loss as the network trains

        for epoch in range(n_epochs):  # loop over the dataset multiple times
            output_epoch = epoch + saved_epoch
            running_loss = 0.0

            for batch_i, data in enumerate(train_loader):
                # get the input images and their corresponding labels
                inputs, labels = data
                inputs, labels = inputs.cuda(), labels.cuda()

                # zero the parameter (weight) gradients
                optimizer.zero_grad()

                # forward pass to get outputs
                outputs = net(inputs, labels)

                # calculate the loss
                # loss = criterion(outputs, labels)
                loss = sum(loss for loss in outputs.values())

                # backward pass to calculate the parameter gradients
                loss.backward()

                # update the parameters
                optimizer.step()

                # print loss statistics
                # to convert loss into a scalar and add it to running_loss, we use .item()
                running_loss += loss.item()

                if batch_i % 45 == 44:    # print every 45 batches
                    avg_loss = running_loss/45
                    # record and print the avg loss over the 100 batches
                    loss_over_time.append(avg_loss)
                    print(f'Epoch: {output_epoch + 1}, Batch: {batch_i+1}, Avg. Loss: {avg_loss}')
                    running_loss = 0.0
            if output_epoch % 100 == 99: # save every 100 epochs
                torch.save(net.state_dict(), f'saved_models/Net2_{output_epoch + 1}.pt')

        print('Finished Training')
        return loss_over_time

    if saved_epoch:
        net.load_state_dict(torch.load(f'saved_models/Net2_{saved_epoch}.pt'))

    # call train and record the loss over time
    training_loss = train(n_epochs)

    # visualize the loss as the network trained
    fig = plt.figure()
    plt.plot(45*np.arange(len(training_loss)), training_loss)
    plt.rc('xtick', labelsize=12)
    plt.rc('ytick', labelsize=12)
    plt.xlabel('Number of Batches', fontsize=12)
    plt.ylabel('loss', fontsize=12)
    plt.ylim(0, 5.5) # consistent scale
    plt.tight_layout()
    if plot_path:
        plt.savefig(os.path.join(plot_path, "Loss_Over_Time"))
        print("saved")
    else:
        plt.show()
    plt.clf()

    # initialize tensor and lists to monitor test loss and accuracy
    test_loss = torch.zeros(1).cuda()
    class_correct = list(0. for i in range(len(classes)))
    class_total = list(0. for i in range(len(classes)))

    # set the module to evaluation mode
    # used to turn off layers that are only useful for training
    # like dropout and batch_norm


    for images, labels in test_loader:

        # get the input images and their corresponding labels
        images, labels = images.cuda(), labels.cuda()

        net.eval()
        # forward pass to get outputs
        with torch.no_grad():
            outputs = net(images)

        # update average test loss
        test_loss += criterion(outputs, labels).item()
        # Get the predicted class
        # Extract the classification scores and apply torch.max
        scores = outputs[0]['scores']  # Assuming outputs[0] contains detection results
        _, predicted = torch.max(scores, 1)
        # count up total number of correct labels
        # for which the predicted and true labels are equal
        total += labels.size(0)
        correct += (predicted == labels).sum()
        # calculate the loss
        # calculate test accuracy for *each* object class
        # we get the scalar value of correct items for a class, by calling `correct[i].item()`
        for l, c in zip(labels.data, correct):
            class_correct[l] += c.item()
            class_total[l] += 1
    accuracy = 100.0 * correct.item() / total



        # compare predictions to true label
        # this creates a `correct` Tensor that holds the number of correctly classified images in a batch




    print(f'Test Loss: {test_loss.cpu().numpy()[0]:.6f}\n')

    for i in range(len(classes)):
        if class_total[i] > 0:
            print('Test Accuracy of %30s: %2d%% (%2d/%2d)' % (
                classes[i], 100 * class_correct[i] / class_total[i],
                np.sum(class_correct[i]), np.sum(class_total[i])))
        else:
            print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))


    print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
        100. * np.sum(class_correct) / np.sum(class_total),
        np.sum(class_correct), np.sum(class_total)))

    # Visualize Sample Results (Runs until a batch contains a )
    # plot the images in the batch, along with predicted and true labels
    fig = plt.figure(figsize=(batch_size/4+5, batch_size/4+5))
    misclassification_found = False
    while(not misclassification_found):
        fig.clf()
        # obtain one batch of test images
        dataiter = iter(test_loader)
        images, labels = dataiter.next()
        images, labels = images.cuda(), labels.cuda()
        # get predictions
        preds = np.squeeze(net(images).data.max(1, keepdim=True)[1].cpu().numpy())
        images = np.swapaxes(np.swapaxes(images.cpu().numpy(), 1, 2), 2, 3)
        for idx in np.arange(batch_size):
            ax = fig.add_subplot(batch_size/8, 8, idx+1, xticks=[], yticks=[])
            ax.imshow(images[idx])
            if preds[idx]==labels[idx]:
                ax.set_title(f"{classes[preds[idx]], classes[labels[idx]]}", color="green")
            else:
                ax.set_title(f"({classes[labels[idx]]})\n{classes[preds[idx]]}", color="red", pad=.4)
                misclassification_found = True
    if plot_path:
        plt.savefig(os.path.join(plot_path, "Results Visualization"))
    else:
        plt.show()
    plt.clf()

In [23]:
def main():
    args = parse_args()
    train_model(batch_size=32,
                n_epochs=20,
                learning_rate = 0.001,
                saved_epoch = 5,
                run_id=args.run_id)
    print("ok")

In [ ]:
import os

def check_image_directories(root_dir):
    """Checks if image directories are populated and contain supported image formats."""

    supported_extensions = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp')

    for subdir in ['train', 'valid', 'test']:
        dir_path = os.path.join(root_dir, subdir)
        if not os.path.exists(dir_path):
            print(f"Directory {dir_path} does not exist.")
            return False

        if not os.listdir(dir_path):
            print(f"Directory {dir_path} is empty.")
            return False

        for item in os.listdir(dir_path):
            item_path = os.path.join(dir_path, item)
            if os.path.isfile(item_path) and not item_path.lower().endswith(supported_extensions):
                print(f"File {item_path} has an unsupported format.")
                return False

    print("Image directories are populated and contain supported image formats.")
    return True

# Call the function to check your image directories
root_dir = './classified_images/'  # Replace with your image directory path
print(check_image_directories(root_dir))

Image directories are populated and contain supported image formats.
True


In [26]:
if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


/content/classified_images directory exists.
Image directories are populated and contain supported image formats.
Classes: ['Afghan_hound', 'African_hunting_dog', 'Airedale', 'American_Staffordshire_terrier', 'Appenzeller', 'Australian_terrier', 'Bedlington_terrier', 'Bernese_mountain_dog', 'Blenheim_spaniel', 'Border_collie', 'Border_terrier', 'Boston_bull', 'Bouvier_des_Flandres', 'Brabancon_griffon', 'Brittany_spaniel', 'Cardigan', 'Chesapeake_Bay_retriever', 'Chihuahua', 'Dandie_Dinmont', 'Doberman', 'English_foxhound', 'English_setter', 'English_springer', 'EntleBucher', 'Eskimo_dog', 'French_bulldog', 'German_shepherd', 'German_short', 'Gordon_setter', 'Great_Dane', 'Great_Pyrenees', 'Greater_Swiss_Mountain_dog', 'Ibizan_hound', 'Irish_setter', 'Irish_terrier', 'Irish_water_spaniel', 'Irish_wolfhound', 'Italian_greyhound', 'Japanese_spaniel', 'Kerry_blue_terrier', 'Labrador_retriever', 'Lakeland_terrier', 'Leonberg', 'Lhasa', 'Maltese_dog', 'Mexican_hairless', 'Newfoundland', 'No

AttributeError: 'list' object has no attribute 'data'

<Figure size 1300x1300 with 0 Axes>